In [1]:
import numpy as np
import pandas as pd
import import_ipynb
import glob

In [2]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ',  'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

abbr_list=list(countries.values())

In [3]:
fuels=['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore']

In [4]:
import matplotlib.pyplot as plt 

# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

In [4]:
import data_preperation as dp

importing Jupyter notebook from data_preperation.ipynb


In [5]:
load_dic = dp.load(countries)[1]
gen_dic = dp.generation(countries)[1]
transmission_data = dp.cross_border(abbr_list)[1]

In [7]:
def sigma(load_data, generation_data, transmission_data, abbr_list):

    eph = 0.1
    A = 100
    load_gen_data = {}
    sigma = {}

    # First we create a new dictionary called 'load_gen_data' and in that dictionary keys are country_abbreviations and as value of each key we add the combined demand column of each country and generation columns from all the sources in that country.
    # Then in each value of the dictionary (which is a dataframe), we add new columns with the name '{Original_column_name} moving_average' and make it 0.
    # Then we get the moving average of each timestep using the values in that same time period in the last 10 days.
    # This creates null values in the first 10 days of the year. We fill them with the values from the original column.
    # Then in these 'moving_average' columns we replace the value with 0.1, if the current value is less than 0.1.
    # Then in these 'moving_average' columns we change the value as 100/current_value.
    # Then we filter only the column names which have 'moving_avareg' as column name and remove the 'moving_average' part from the column name.
    # We save the resultant dataframe as a value in a dictionary called 'sigma' with the key as country_abbreviation.
    # We do the same procedure for transmission_data and save the resultant dataframe as the value of 'transmission_data' key of 'sigma' dictionary.

    for abbr, df in generation_data.items():
        load_gen_data[abbr] = pd.concat([df, load_data[abbr]], axis=1)

    for abbr, df in load_gen_data.items():
        for column in df.columns.values:
            column_data = [[index, value]
                           for index, value in enumerate(df.loc[:, column])]
            for index, value in column_data:
                if pd.isnull(value):
                    temp_list = [[abs(index-item[0]), item[1]]
                                 for item in column_data if (index-item[0]) % 24 == 0]
                    temp_list.sort()
                    moving_average = [x[1] for x in temp_list[1:11]]
                    moving_average = [0 if pd.isna(
                        x) else x for x in moving_average]
                    df.loc[index, column] = np.mean(np.array(moving_average))
            df[column] = df[column].apply(lambda x: eph if x < eph else x)
            df[column] = df[column].apply(lambda x:  A/x)
        sigma[abbr] = df

    for column in transmission_data.columns.values:
        column_data = [[index, value] for index,
                       value in enumerate(transmission_data.loc[:, column])]
        for index, value in column_data:
            if pd.isnull(value):
                temp_list = [[abs(index-item[0]), item[1]]
                             for item in column_data if (index-item[0]) % 24 == 0]
                temp_list.sort()
                moving_average = [x[1] for x in temp_list[1:11]]
                moving_average = [0 if pd.isna(
                    x) else x for x in moving_average]
                transmission_data.loc[index, column] = np.mean(
                    np.array(moving_average))
        transmission_data[column] = transmission_data[column].apply(
            lambda x: eph if x < eph else x)
        transmission_data[column] = transmission_data[column].apply(
            lambda x:  A/x)
    sigma["transmission_data"] = transmission_data

    return(sigma)


In [9]:
sigma = sigma(load_dic,gen_dic,transmission_data,abbr_list)

In [12]:
for key,value in sigma.items():
    print(key)
    for column in value.columns.values:
        print(column,value[column].isnull().sum())

AT
0
0
0
0
0
0
0
0
0
0
0
BE
0
0
0
0
0
0
0
0
0
0
0
0
BG
0
0
0
0
0
0
0
0
0
0
0
0
CH
0
0
0
0
0
0
0
CZ
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
DE
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
DK
0
0
0
0
0
0
0
0
0
EE
0
0
0
0
0
0
0
0
0
0
0
ES
0
0
0
0
0
0
0
0
0
0
0
0
0
0
FI
0
0
0
0
0
0
0
0
0
0
0
0
FR
0
0
0
0
0
0
0
0
0
0
0
0
GR
0
0
0
0
0
HU
0
0
0
0
0
0
0
0
0
0
0
0
IE
0
0
0
0
0
0
0
0
0
IT
0
0
0
0
0
0
0
0
0
0
0
0
0
0
LT
0
0
0
0
0
0
0
0
0
LV
0
0
0
0
0
0
ME
0
0
0
0
NL
0
0
0
0
0
0
0
0
0
0
NO
0
0
0
0
0
0
PL
0
0
0
0
0
0
0
0
0
0
0
PT
0
0
0
0
0
0
0
0
0
0
RS
0
0
0
0
0
0
0
0
SE
0
0
0
0
0
SI
0
0
0
0
0
0
0
0
0
0
0
SK
0
0
0
0
0
0
0
0
0
0
0
0
0
UK
0
0
0
0
0
0
0
0
0
0
0
0
transmission_data
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
